In [1]:
import asyncio
import websockets
import json

import ccxt

In [4]:
websocket_base_url = "wss://stream.binance.com:9443"
websocket_base_url_alt = "wss://data-stream.binance.vision"

In [105]:
interval = "1h"
token = "BTCUSDT"

In [ ]:
def fetch_token_data_stream(token):
    url = f"{websocket_base_url}/ws/{token.lower()}@kline_{interval}"
    async with websockets.connect(url) as websocket:
        while True:
            response = await websocket.recv()
            data = json.loads(response)
            print(data)

In [ ]:




async def fetch_binance_btcusdt():
    url = "wss://stream.binance.com:9443/ws/btcusdt@trade"
    async with websockets.connect(url) as websocket:
        while True:
            response = await websocket.recv()
            data = json.loads(response)
            print(data)


# Run the function
asyncio.get_event_loop().run_until_complete(fetch_binance_btcusdt())

In [34]:
def get_top_50_mcap_assets():
    binance = ccxt.binance()
    markets = binance.load_markets()
    tickers = binance.fetch_tickers()

    # Filter out only USDT pairs and sort by market cap
    usdt_pairs = {
        symbol: ticker for symbol, ticker in tickers.items() if symbol.endswith("/USDT")
    }
    sorted_pairs = sorted(
        usdt_pairs.items(), key=lambda item: item[1]["quoteVolume"], reverse=True
    )

    # Get the top 50 pairs
    top_50_pairs = sorted_pairs[:50]

    return {pair[1]["info"]["symbol"]: pair[1]["quoteVolume"] for pair in top_50_pairs}


# Run the function
top_50_tokens = get_top_50_mcap_assets()

In [ ]:
binance_base_url = "https://api3.binance.com/api/v3"
top_50_tokens.keys()

In [79]:
import requests
import pandas as pd
from datetime import datetime, timedelta


def get_binance_klines(symbol, interval, start_time, limit=1000):
    url = f"{binance_base_url}/klines"
    params = {
        "symbol": symbol,
        "interval": interval,
        "startTime": int(start_time.timestamp() * 1000),
        "limit": limit,
    }
    response = requests.get(url, params=params)
    data = response.json()
    data = pd.DataFrame(
        data,
        columns=[
            "open_time",
            "open",
            "high",
            "low",
            "close",
            "volume",
            "timestamp",
            "quote_av",
            "trades",
            "tb_base_av",
            "tb_quote_av",
            "ignore",
        ],
    )

    return data.loc[:, ["timestamp", "open", "high", "low", "close", "volume"]]

In [80]:
# Define the time range for the last 2 years
end_time = datetime.now()
start_time = end_time - timedelta(days=2 * 365)

In [ ]:
get_binance_klines("BTCUSDT", "1h", start_time)

In [ ]:
# Fetch kline data for each symbol in top_50_tokens
all_klines_data = {}
for symbol in top_50_tokens.keys():
    klines_data = get_binance_klines(symbol, "1h", start_time, end_time)
    all_klines_data[symbol] = klines_data

In [ ]:
datetime.fromtimestamp(all_klines_data["BTCUSDT"].timestamp.astype(int) // 1000)

In [ ]:
pd.TimedeltaIndex(all_klines_data["BTCUSDT"].timestamp, unit="ms")

In [ ]:
pd.to_datetime(all_klines_data["BTCUSDT"].timestamp, unit="ms")

## get data from binance

### imports

In [8]:
import pandas as pd
import os
import requests
import zipfile
from io import BytesIO
import ccxt
import sqlite3
import aiosqlite
from tqdm import tqdm
import multiprocessing as mp

### parameters

In [9]:
data_url = "https://data.binance.vision/data/spot/monthly/klines/{ticker}/{interval}/{ticker}-{interval}-{date}.zip"
interval = "1h"
date_range = (
    pd.date_range(start="2021-01", end="2024-12", freq="MS").strftime("%Y-%m").tolist()
)


In [ ]:
def get_top_50_mcap_assets():
    binance = ccxt.binance()
    tickers = binance.fetch_tickers()

    # Filter out only USDT pairs and sort by market cap
    usdt_pairs = {
        symbol: ticker for symbol, ticker in tickers.items() if symbol.endswith("/USDT")
    }
    sorted_pairs = sorted(
        usdt_pairs.items(), key=lambda item: item[1]["quoteVolume"], reverse=True
    )

    # Get the top 50 pairs
    top_50_pairs = sorted_pairs[:50]
    return top_50_pairs

    # return {pair[1]["info"]["symbol"]: pair[1]["quoteVolume"] for pair in top_50_pairs}


# Run the function
top_50_tokens = get_top_50_mcap_assets()

# top_50_tokens.keys()

AttributeError: 'list' object has no attribute 'keys'

In [12]:
top_50_tokens[0]

('BTC/USDT',
 {'symbol': 'BTC/USDT',
  'timestamp': 1736608909507,
  'datetime': '2025-01-11T15:21:49.507Z',
  'high': 95836.0,
  'low': 92957.86,
  'bid': 94439.58,
  'bidVolume': 5.8846,
  'ask': 94439.59,
  'askVolume': 0.02254,
  'vwap': 94453.69105648,
  'open': 93096.08,
  'close': 94439.58,
  'last': 94439.58,
  'previousClose': 93096.2,
  'change': 1343.5,
  'percentage': 1.443,
  'average': 93767.83,
  'baseVolume': 15972.36036,
  'quoteVolume': 1508648390.8861551,
  'markPrice': None,
  'indexPrice': None,
  'info': {'symbol': 'BTCUSDT',
   'priceChange': '1343.50000000',
   'priceChangePercent': '1.443',
   'weightedAvgPrice': '94453.69105648',
   'prevClosePrice': '93096.20000000',
   'lastPrice': '94439.58000000',
   'lastQty': '0.00054000',
   'bidPrice': '94439.58000000',
   'bidQty': '5.88460000',
   'askPrice': '94439.59000000',
   'askQty': '0.02254000',
   'openPrice': '93096.08000000',
   'highPrice': '95836.00000000',
   'lowPrice': '92957.86000000',
   'volume': '

In [4]:
# Create a dataframe from the top_50_tokens dictionary
df_top_50_tokens = pd.DataFrame(
    list(top_50_tokens.items()), columns=["Token", "Market Cap"]
)

df_top_50_tokens.head()

,Token,Market Cap
0,BTCUSDT,1.999356e+09
1,ETHUSDT,1.011797e+09
2,SOLUSDT,5.829927e+08
3,XRPUSDT,5.532881e+08
4,USDCUSDT,5.513838e+08


In [5]:
# crossproduct of tokens and date range
cross_product = [
    data_url.format(ticker=ticker, interval=interval, date=date)
    for ticker in df_top_50_tokens["Token"]
    for date in date_range
]

In [6]:
# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect("data.db")
cursor = conn.cursor()

# Create a table with the specified columns
cursor.execute("""
CREATE TABLE IF NOT EXISTS token_prices (
    token_name TEXT,
    timestamp DATETIME,
    open REAL,
    high REAL,
    low REAL,
    close REAL,
    volume REAL
)
""")

# Commit the changes and close the connection
conn.commit()
conn.close()

In [7]:
failed_tokens = []
failed_urls = []

In [11]:
def get_price_df(url):
    file_name = f"{url.split('/')[-1].split('.')[0]}.csv"
    token = url.split("/")[-2]

    # Download and extract the data if it doesn't exist
    if not os.path.exists(f"./data/{file_name}"):
        response = requests.get(url)
        if response.status_code == 200:
            with zipfile.ZipFile(BytesIO(response.content)) as z:
                z.extractall("data")
            # print(f"Downloaded and extracted data for {url}")
        else:
            print(f"Failed to download data for {url}")
            failed_urls.append(url)
            failed_tokens.append(token)

    df = (
        pd.read_csv(f"./data/{file_name}", header=None)
        .iloc[:, [6, 1, 2, 3, 4, 5]]
        .rename(
            columns={
                6: "timestamp",
                1: "open",
                2: "high",
                3: "low",
                4: "close",
                5: "volume",
            }
        )
    )

    df = df.astype({
        "timestamp": "datetime64[ms]",
        "open": "float64",
        "high": "float64",
        "low": "float64",
        "close": "float64",
        "volume": "float64",
    })

    df["token"] = url.split("/")[-1].split("-")[0]

    # print(f"Inserted data for {url} into SQLite database")

    return df

In [ ]:
import concurrent.futures

# Define the number of workers to use
num_workers = 8

# Use ThreadPoolExecutor to download data concurrently
with concurrent.futures.ThreadPoolExecutor(max_workers=num_workers) as executor:
    # Map the get_price_df function to the cross_product list
    price_data_df = list(
        tqdm(executor.map(get_price_df, cross_product), total=len(cross_product))
    )

In [ ]:
price_data_df

In [ ]:
await get_price_df(cross_product[0])

In [ ]:
 async with aiosqlite.connect("data.db") as db:
            await db.executemany(
                """
                INSERT INTO token_prices (token_name, timestamp, open, high, low, close, volume)
                VALUES (?, ?, ?, ?, ?, ?, ?)
                """,
                df[
                    ["token", "timestamp", "open", "high", "low", "close", "volume"]
                ].values.tolist(),
            )
            await db.commit()